## Comment Analysis

### Read Data

In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import string
import re
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer

In [88]:
data_raw = pd.read_csv('item_cmt.csv')
data_raw

,shopid,itemid,rating_star,cmt
0,4937640,206979605,5,Good product quality
1,4937640,18982557,4,The thing came on time however it was broken t...
2,4149345,195378126,3,Product quality is ok Delivery time as expect...
3,4937640,40486732,1,"Delivered as a defect item, with chipped end."
4,4937640,282750352,5,Good product quality
5,4937640,127478636,5,Good product quality Fast delivery Excellent ...
6,4937640,16804582,4,"Delivery and seller, good. Product is ok? It's..."
7,4149345,14479514,5,Good product quality Fast delivery Excellent ...
8,4149345,14482633,5,Trusted seller
9,4937640,22563846,5,Good product quality


In [95]:
data_raw = data_raw.drop_duplicates()
data_raw = data_raw[data_raw["cmt"].str.contains("Good product quality") == False]
data_raw = data_raw[data_raw["cmt"].str.contains("Fast delivery") == False]
data_raw = data_raw[data_raw["cmt"].str.contains("Excellent Service") == False]
data_raw.to_csv('filter_data.csv',header=True)

In [96]:
data = pd.read_csv('filter_data.csv')
data

,Unnamed: 0,shopid,itemid,rating_star,cmt,Sentiment
0,1,4937640,18982557,4,The thing came on time however it was broken t...,B
1,2,4149345,195378126,3,Product quality is ok Delivery time as expect...,G
2,3,4937640,40486732,1,"Delivered as a defect item, with chipped end.",B
3,6,4937640,16804582,4,"Delivery and seller, good. Product is ok? It's...",G
4,8,4149345,14482633,5,Trusted seller,G
5,10,4149345,13434358,0,Where's it made in?,N
6,11,4149345,42089797,5,"Same as photo,but wire got yellowish,box look ...",B
7,22,4937640,43305088,4,Got about 96 pieces in total but fair enough f...,G
8,23,4149345,60228536,3,Product quality is ok,G
9,25,4149345,16431297,3,Not compatible with Xbox OG,G


In [98]:
data = data[data["rating_star"]!=3]
data

,Unnamed: 0,shopid,itemid,rating_star,cmt,Sentiment
0,1,4937640,18982557,4,The thing came on time however it was broken t...,B
2,3,4937640,40486732,1,"Delivered as a defect item, with chipped end.",B
3,6,4937640,16804582,4,"Delivery and seller, good. Product is ok? It's...",G
4,8,4149345,14482633,5,Trusted seller,G
5,10,4149345,13434358,0,Where's it made in?,N
6,11,4149345,42089797,5,"Same as photo,but wire got yellowish,box look ...",B
7,22,4937640,43305088,4,Got about 96 pieces in total but fair enough f...,G
10,27,4149345,46508803,5,It works well! Item came earlier than xpected ...,G
11,28,4149345,14477372,5,Smaller than expected but still great. Not sur...,B
12,29,4937640,18981664,5,Tested working but yet to try on hair.,N


In [99]:
target = data["Sentiment"]
cmt = data["cmt"]

### Data Preprocessing

In [100]:
def tokenize(text):
    stemmer = PorterStemmer()
    
    text = re.sub(r"[^a-zA-Z]"," ",text)
    
    tokens = word_tokenize(text)
    tokens = [i for i in tokens if i not in string.punctuation]
    
    stems = stem_tokens(tokens, stemmer)
    return stems

def stem_tokens(t,s):
    stemmed=[]
    for item in t:
        stemmed.append(s.stem(item))
    return stemmed

In [101]:
vectorizer = TfidfVectorizer(stop_words='english', tokenizer = tokenize)
cmt_tfidf = vectorizer.fit_transform(cmt)

In [102]:
train, test, train_target,test_target = train_test_split(cmt_tfidf, target, test_size = 0.3, random_state = 0)

In [111]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

model = RandomForestClassifier()
model.fit(train, train_target)
pred = model.predict(test)
print "Random Forest: %f" %accuracy_score(test_target, pred)


model = LogisticRegression()
model.fit(train, train_target)
pred = model.predict(test)
print "Logistic Regression: %f" %accuracy_score(test_target, pred)

model = AdaBoostClassifier()
model.fit(train, train_target)
pred = model.predict(test)
print "Adaboost: %f" %accuracy_score(test_target, pred)

model = ExtraTreesClassifier()
model.fit(train, train_target)
pred = model.predict(test)
print "Extra tree: %f" %accuracy_score(test_target, pred)

model = KNeighborsClassifier(n_neighbors=3)
model.fit(train, train_target)
pred = model.predict(test)
print "5 Nearest Neighbors: %f" %accuracy_score(test_target, pred)

model = MLPClassifier()
model.fit(train, train_target)
pred = model.predict(test)
print "Multilayer Perceptron: %f" %accuracy_score(test_target, pred)

Random Forest: 0.763795
Logistic Regression: 0.808325
Adaboost: 0.745402
Extra tree: 0.774443
5 Nearest Neighbors: 0.606002
Multilayer Perceptron: 0.760891


In [104]:
model = LogisticRegression()
model.get_params().keys()

['warm_start',
 'C',
 'n_jobs',
 'verbose',
 'intercept_scaling',
 'fit_intercept',
 'max_iter',
 'penalty',
 'multi_class',
 'random_state',
 'dual',
 'tol',
 'solver',
 'class_weight']

In [112]:
# tuned_parameters = {'hidden_layer_sizes':[(100,),(50,50),(30,30,30)], 'activation':['identity','logistic','tanh','relu'],
#                     'solver':['lbfgs','sgd','adam'],'alpha':[0.01,0.0001,0.000001], 'batch_size':[50,100,200],
#                     'learning_rate':['constant','invscaling','adaptive']}
tuned_parameters = {'penalty':['l2'],'class_weight':[None,'balanced'],'solver':['newton-cg','lbfgs','liblinear','sag'], 'warm_start':[True,False]}
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(LogisticRegression(), tuned_parameters, cv=5)
clf.fit(train, train_target)
print clf.best_params_

{'penalty': 'l2', 'warm_start': True, 'solver': 'liblinear', 'class_weight': 'balanced'}


In [119]:
pred = clf.predict(test)
print "Multilayer Perceptron: %f" %accuracy_score(test_target, pred)
print classification_report(test_target, pred)

Multilayer Perceptron: 0.801549
             precision    recall  f1-score   support

          B       0.75      0.72      0.73       218
          G       0.90      0.85      0.88       615
          N       0.61      0.72      0.67       200

avg / total       0.81      0.80      0.80      1033

